# Azure AI Vision Services Demo

This notebook demonstrates the capabilities of Azure AI Vision Services including:
- **Image Analysis**: Describe images, detect objects, and extract visual features
- **Optical Character Recognition (OCR)**: Extract text from images and documents
- **Face Detection**: Detect and analyze faces in images
- **Custom Vision**: Train custom image classification models
- **Content Moderation**: Detect inappropriate content in images

## Prerequisites
- Azure subscription
- Azure Computer Vision resource created in Azure portal
- Python 3.8 or higher
- Sample images for testing

## 1. Setup and Package Installation

First, let's install the required Azure Computer Vision SDK and supporting packages.

In [ ]:
# Install required Azure Computer Vision SDK and supporting packages
!pip install azure-cognitiveservices-vision-computervision azure-identity python-dotenv
!pip install pillow matplotlib requests numpy opencv-python
!pip install azure-ai-vision-imageanalysis  # Latest Vision SDK

# For image processing and visualization
!pip install IPython

In [ ]:
# Import required libraries
import os
import io
import json
import requests
from dotenv import load_dotenv
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from IPython.display import display, Image as IPImage

# Azure Computer Vision imports
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully!")
print("📦 Ready to analyze images with Azure AI Vision!")

## 2. Azure Computer Vision Configuration

### Option 1: Using Environment Variables (Recommended)
Set these environment variables in your system or create a `.env` file:
```
AZURE_VISION_ENDPOINT=https://your-resource.cognitiveservices.azure.com/
AZURE_VISION_KEY=your-api-key
```

### Option 2: Using Managed Identity (For Azure-hosted applications)
When running on Azure services with managed identity enabled.

In [ ]:
# Configuration for Azure Computer Vision Service
vision_endpoint = os.getenv('AZURE_VISION_ENDPOINT') or "https://your-resource.cognitiveservices.azure.com/"
vision_key = os.getenv('AZURE_VISION_KEY') or "your-vision-key-here"

# Create Computer Vision client
if vision_key and vision_key != "your-vision-key-here":
    credential = AzureKeyCredential(vision_key)
    vision_client = ComputerVisionClient(endpoint=vision_endpoint, credentials=credential)
    print(f"✅ Computer Vision client created successfully!")
    print(f"🔗 Endpoint: {vision_endpoint}")
else:
    print("❌ Please set AZURE_VISION_ENDPOINT and AZURE_VISION_KEY environment variables")
    print("You can get these from your Azure Computer Vision resource in the Azure portal")

# Helper function to load and display images
def load_and_display_image(image_path_or_url, title="Image", figsize=(10, 8)):
    """
    Load and display an image from local path or URL
    """
    try:
        if image_path_or_url.startswith('http'):
            response = requests.get(image_path_or_url)
            image = Image.open(io.BytesIO(response.content))
        else:
            image = Image.open(image_path_or_url)
        
        plt.figure(figsize=figsize)
        plt.imshow(image)
        plt.title(title)
        plt.axis('off')
        plt.show()
        return image
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return None

In [ ]:
# Download sample images for testing
import urllib.request

sample_images = {
    "landmark.jpg": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/landmark.jpg",
    "objects.jpg": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/objects.jpg",
    "faces.jpg": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/faces.jpg",
    "text.jpg": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/printed_text.jpg"
}

def download_sample_images():
    """Download sample images for testing"""
    print("📥 Downloading sample images...")
    
    # Create images directory if it doesn't exist
    os.makedirs("sample_images", exist_ok=True)
    
    for filename, url in sample_images.items():
        filepath = os.path.join("sample_images", filename)
        if not os.path.exists(filepath):
            try:
                urllib.request.urlretrieve(url, filepath)
                print(f"✅ Downloaded: {filename}")
            except Exception as e:
                print(f"❌ Failed to download {filename}: {e}")
        else:
            print(f"📁 Already exists: {filename}")
    
    print("🎯 Sample images ready for analysis!")

# Download sample images
download_sample_images()

## 3. Image Analysis and Description

Azure Computer Vision can analyze images and provide:
- **Descriptions**: Natural language descriptions of image content
- **Tags**: Relevant tags for the image
- **Categories**: Image categorization
- **Objects**: Object detection with bounding boxes
- **Brands**: Brand detection
- **Faces**: Face detection and analysis

In [ ]:
def analyze_image_comprehensive(image_path_or_url):
    """
    Perform comprehensive image analysis
    
    Args:
        image_path_or_url (str): Path to local image or URL
    
    Returns:
        dict: Analysis results
    """
    try:
        print(f"🔍 Analyzing image: {image_path_or_url}")
        
        # Display the image
        image = load_and_display_image(image_path_or_url, "Image for Analysis")
        if not image:
            return None
        
        # Define visual features to extract
        features = [
            VisualFeatureTypes.description,
            VisualFeatureTypes.tags,
            VisualFeatureTypes.categories,
            VisualFeatureTypes.objects,
            VisualFeatureTypes.brands,
            VisualFeatureTypes.faces,
            VisualFeatureTypes.adult,
            VisualFeatureTypes.color
        ]
        
        # Analyze image
        if image_path_or_url.startswith('http'):
            analysis = vision_client.analyze_image(image_path_or_url, visual_features=features)
        else:
            with open(image_path_or_url, 'rb') as image_stream:
                analysis = vision_client.analyze_image_in_stream(image_stream, visual_features=features)
        
        # Process and display results
        results = {}
        
        # Description
        if analysis.description:
            print("\n📝 DESCRIPTION:")
            if analysis.description.captions:
                for caption in analysis.description.captions:
                    print(f"   📖 {caption.text} (confidence: {caption.confidence:.2f})")
                    results['description'] = caption.text
        
        # Tags
        if analysis.tags:
            print("\n🏷️ TAGS:")
            tags = []
            for tag in analysis.tags:
                if tag.confidence > 0.5:  # Filter by confidence
                    print(f"   🔖 {tag.name} (confidence: {tag.confidence:.2f})")
                    tags.append({'name': tag.name, 'confidence': tag.confidence})
            results['tags'] = tags
        
        # Categories
        if analysis.categories:
            print("\n📂 CATEGORIES:")
            categories = []
            for category in analysis.categories:
                print(f"   📁 {category.name} (score: {category.score:.2f})")
                categories.append({'name': category.name, 'score': category.score})
            results['categories'] = categories
        
        # Objects
        if analysis.objects:
            print("\n🎯 OBJECTS DETECTED:")
            objects = []
            for obj in analysis.objects:
                print(f"   🔍 {obj.object_property} (confidence: {obj.confidence:.2f})")
                print(f"      📍 Location: ({obj.rectangle.x}, {obj.rectangle.y}, {obj.rectangle.w}, {obj.rectangle.h})")
                objects.append({
                    'name': obj.object_property,
                    'confidence': obj.confidence,
                    'rectangle': {
                        'x': obj.rectangle.x,
                        'y': obj.rectangle.y,
                        'w': obj.rectangle.w,
                        'h': obj.rectangle.h
                    }
                })
            results['objects'] = objects
        
        # Faces
        if analysis.faces:
            print("\n👤 FACES DETECTED:")
            faces = []
            for face in analysis.faces:
                print(f"   👤 Age: {face.age}, Gender: {face.gender}")
                print(f"      📍 Location: ({face.face_rectangle.left}, {face.face_rectangle.top}, {face.face_rectangle.width}, {face.face_rectangle.height})")
                faces.append({
                    'age': face.age,
                    'gender': face.gender.value if face.gender else None,
                    'rectangle': {
                        'left': face.face_rectangle.left,
                        'top': face.face_rectangle.top,
                        'width': face.face_rectangle.width,
                        'height': face.face_rectangle.height
                    }
                })
            results['faces'] = faces
        
        # Color analysis
        if analysis.color:
            print("\n🎨 COLOR ANALYSIS:")
            print(f"   🎨 Dominant background color: {analysis.color.dominant_color_background}")
            print(f"   🎨 Dominant foreground color: {analysis.color.dominant_color_foreground}")
            print(f"   🎨 Accent color: #{analysis.color.accent_color}")
            print(f"   ⚫ Is black and white: {analysis.color.is_bw_img}")
            results['color'] = {
                'background': analysis.color.dominant_color_background,
                'foreground': analysis.color.dominant_color_foreground,
                'accent': analysis.color.accent_color,
                'is_bw': analysis.color.is_bw_img
            }
        
        return results
        
    except Exception as e:
        print(f"❌ Error analyzing image: {e}")
        return None

# Test image analysis with sample images
for filename in ["landmark.jpg", "objects.jpg", "faces.jpg"]:
    filepath = os.path.join("sample_images", filename)
    if os.path.exists(filepath):
        print(f"\n" + "="*80)
        print(f"🖼️ ANALYZING: {filename}")
        print("="*80)
        results = analyze_image_comprehensive(filepath)
    else:
        print(f"❌ Sample image not found: {filepath}")

In [ ]:
def visualize_objects_and_faces(image_path, analysis_results):
    """
    Visualize detected objects and faces with bounding boxes
    
    Args:
        image_path (str): Path to the image
        analysis_results (dict): Results from image analysis
    """
    try:
        # Load image
        if image_path.startswith('http'):
            response = requests.get(image_path)
            image = Image.open(io.BytesIO(response.content))
        else:
            image = Image.open(image_path)
        
        # Create figure with subplots
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))
        
        # Original image
        axes[0].imshow(image)
        axes[0].set_title("Original Image")
        axes[0].axis('off')
        
        # Image with annotations
        axes[1].imshow(image)
        axes[1].set_title("Detected Objects and Faces")
        axes[1].axis('off')
        
        # Draw object bounding boxes
        if 'objects' in analysis_results and analysis_results['objects']:
            for obj in analysis_results['objects']:
                rect = obj['rectangle']
                # Create rectangle patch
                bbox = patches.Rectangle(
                    (rect['x'], rect['y']), 
                    rect['w'], 
                    rect['h'],
                    linewidth=2, 
                    edgecolor='red', 
                    facecolor='none'
                )
                axes[1].add_patch(bbox)
                
                # Add label
                axes[1].text(
                    rect['x'], 
                    rect['y'] - 5, 
                    f"{obj['name']} ({obj['confidence']:.2f})",
                    fontsize=10, 
                    color='red', 
                    bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8)
                )
        
        # Draw face bounding boxes
        if 'faces' in analysis_results and analysis_results['faces']:
            for face in analysis_results['faces']:
                rect = face['rectangle']
                # Create rectangle patch
                bbox = patches.Rectangle(
                    (rect['left'], rect['top']), 
                    rect['width'], 
                    rect['height'],
                    linewidth=2, 
                    edgecolor='blue', 
                    facecolor='none'
                )
                axes[1].add_patch(bbox)
                
                # Add label
                label = f"Age: {face['age']}, {face['gender']}"
                axes[1].text(
                    rect['left'], 
                    rect['top'] - 5, 
                    label,
                    fontsize=10, 
                    color='blue', 
                    bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8)
                )
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Error visualizing objects: {e}")

# Visualize objects for sample images with detection results
for filename in ["objects.jpg", "faces.jpg"]:
    filepath = os.path.join("sample_images", filename)
    if os.path.exists(filepath):
        print(f"\n🎨 VISUALIZING DETECTIONS: {filename}")
        results = analyze_image_comprehensive(filepath)
        if results:
            visualize_objects_and_faces(filepath, results)

## 4. Optical Character Recognition (OCR)

OCR extracts text from images and documents. Azure Computer Vision provides:
- **Read API**: Advanced OCR for printed and handwritten text
- **Text extraction**: From images, PDFs, and documents
- **Multiple languages**: Support for 100+ languages
- **Layout preservation**: Maintains text structure and formatting

In [ ]:
def extract_text_from_image(image_path_or_url):
    """
    Extract text from image using OCR
    
    Args:
        image_path_or_url (str): Path to local image or URL
    
    Returns:
        list: Extracted text lines with confidence scores
    """
    try:
        print(f"📄 Extracting text from: {image_path_or_url}")
        
        # Display the image
        image = load_and_display_image(image_path_or_url, "Image for OCR")
        if not image:
            return None
        
        # Start text recognition
        if image_path_or_url.startswith('http'):
            read_response = vision_client.read(image_path_or_url, raw=True)
        else:
            with open(image_path_or_url, 'rb') as image_stream:
                read_response = vision_client.read_in_stream(image_stream, raw=True)
        
        # Get operation location
        read_operation_location = read_response.headers["Operation-Location"]
        operation_id = read_operation_location.split("/")[-1]
        
        # Wait for the operation to complete
        print("⏳ Processing text recognition...")
        while True:
            read_result = vision_client.get_read_result(operation_id)
            if read_result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)
        
        # Extract text results
        extracted_text = []
        if read_result.status == OperationStatusCodes.succeeded:
            print("\n✅ TEXT EXTRACTION RESULTS:")
            
            for text_result in read_result.analyze_result.read_results:
                for line in text_result.lines:
                    print(f"📝 Text: '{line.text}'")
                    print(f"📊 Confidence: {line.appearance.style.confidence if line.appearance and line.appearance.style else 'N/A'}")
                    
                    # Extract bounding box coordinates
                    bbox = line.bounding_box
                    print(f"📍 Bounding box: {bbox}")
                    
                    extracted_text.append({
                        'text': line.text,
                        'bounding_box': bbox,
                        'confidence': line.appearance.style.confidence if line.appearance and line.appearance.style else None
                    })
                    print("-" * 50)
            
            # Show all extracted text combined
            all_text = " ".join([item['text'] for item in extracted_text])
            print(f"\n📖 COMBINED TEXT:\n{all_text}")
            
        else:
            print(f"❌ Text extraction failed: {read_result.status}")
            return None
        
        return extracted_text
        
    except Exception as e:
        print(f"❌ Error extracting text: {e}")
        return None

import time

# Test OCR with sample text image
text_image_path = os.path.join("sample_images", "text.jpg")
if os.path.exists(text_image_path):
    print("\n" + "="*80)
    print("📄 OPTICAL CHARACTER RECOGNITION (OCR)")
    print("="*80)
    ocr_results = extract_text_from_image(text_image_path)
else:
    print("❌ Sample text image not found")

In [ ]:
def visualize_extracted_text(image_path, ocr_results):
    """
    Visualize extracted text with bounding boxes
    
    Args:
        image_path (str): Path to the image
        ocr_results (list): OCR results with bounding boxes
    """
    try:
        # Load image
        if image_path.startswith('http'):
            response = requests.get(image_path)
            image = Image.open(io.BytesIO(response.content))
        else:
            image = Image.open(image_path)
        
        # Create figure
        plt.figure(figsize=(15, 10))
        plt.imshow(image)
        plt.title("Extracted Text with Bounding Boxes")
        plt.axis('off')
        
        # Draw bounding boxes for each text line
        if ocr_results:
            for i, text_item in enumerate(ocr_results):
                bbox = text_item['bounding_box']
                
                # Convert bounding box to rectangle coordinates
                # Bounding box format: [x1, y1, x2, y2, x3, y3, x4, y4]
                if len(bbox) >= 8:
                    x_coords = [bbox[i] for i in range(0, 8, 2)]
                    y_coords = [bbox[i] for i in range(1, 8, 2)]
                    
                    # Create polygon patch
                    polygon = patches.Polygon(
                        list(zip(x_coords, y_coords)),
                        linewidth=2,
                        edgecolor='red',
                        facecolor='none'
                    )
                    plt.gca().add_patch(polygon)
                    
                    # Add text label
                    plt.text(
                        min(x_coords), 
                        min(y_coords) - 10, 
                        f"Line {i+1}",
                        fontsize=10, 
                        color='red', 
                        bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8)
                    )
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Error visualizing text: {e}")

# Visualize OCR results if available
if 'ocr_results' in locals() and ocr_results:
    print("\n🎨 VISUALIZING EXTRACTED TEXT:")
    visualize_extracted_text(text_image_path, ocr_results)

## 5. Smart Thumbnail Generation

Azure Computer Vision can generate smart thumbnails that focus on the most important part of an image.

In [ ]:
def generate_smart_thumbnail(image_path_or_url, width=200, height=200, smart_cropping=True):
    """
    Generate a smart thumbnail from an image
    
    Args:
        image_path_or_url (str): Path to local image or URL
        width (int): Thumbnail width
        height (int): Thumbnail height
        smart_cropping (bool): Enable smart cropping
    
    Returns:
        bytes: Thumbnail image data
    """
    try:
        print(f"🖼️ Generating smart thumbnail ({width}x{height}) from: {image_path_or_url}")
        
        # Generate thumbnail
        if image_path_or_url.startswith('http'):
            thumbnail_stream = vision_client.generate_thumbnail(
                width, height, image_path_or_url, smart_cropping
            )
        else:
            with open(image_path_or_url, 'rb') as image_stream:
                thumbnail_stream = vision_client.generate_thumbnail_in_stream(
                    width, height, image_stream, smart_cropping
                )
        
        # Save and display thumbnail
        thumbnail_data = thumbnail_stream.read()
        thumbnail_filename = f"thumbnail_{width}x{height}_{'smart' if smart_cropping else 'center'}.jpg"
        
        with open(thumbnail_filename, 'wb') as thumbnail_file:
            thumbnail_file.write(thumbnail_data)
        
        # Display original and thumbnail side by side
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        # Original image
        if image_path_or_url.startswith('http'):
            response = requests.get(image_path_or_url)
            original_image = Image.open(io.BytesIO(response.content))
        else:
            original_image = Image.open(image_path_or_url)
        
        axes[0].imshow(original_image)
        axes[0].set_title("Original Image")
        axes[0].axis('off')
        
        # Thumbnail
        thumbnail_image = Image.open(io.BytesIO(thumbnail_data))
        axes[1].imshow(thumbnail_image)
        axes[1].set_title(f"Smart Thumbnail ({width}x{height})")
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"✅ Thumbnail saved as: {thumbnail_filename}")
        return thumbnail_data
        
    except Exception as e:
        print(f"❌ Error generating thumbnail: {e}")
        return None

# Test thumbnail generation
landmark_path = os.path.join("sample_images", "landmark.jpg")
if os.path.exists(landmark_path):
    print("\n" + "="*60)
    print("🖼️ SMART THUMBNAIL GENERATION")
    print("="*60)
    
    # Generate different sizes
    sizes = [(150, 150), (200, 100), (100, 200)]
    
    for width, height in sizes:
        thumbnail = generate_smart_thumbnail(landmark_path, width, height)
else:
    print("❌ Landmark image not found for thumbnail generation")

## 6. Domain-Specific Analysis

Azure Computer Vision provides specialized analysis for:
- **Landmarks**: Famous landmarks and tourist attractions
- **Celebrities**: Celebrity face recognition
- **Domain models**: Specialized models for specific content types

In [ ]:
def analyze_domain_specific(image_path_or_url, domain="landmarks"):
    """
    Perform domain-specific analysis (landmarks or celebrities)
    
    Args:
        image_path_or_url (str): Path to local image or URL
        domain (str): 'landmarks' or 'celebrities'
    
    Returns:
        dict: Domain-specific analysis results
    """
    try:
        print(f"🏛️ Analyzing for {domain}: {image_path_or_url}")
        
        # Display the image
        image = load_and_display_image(image_path_or_url, f"Image for {domain} analysis")
        if not image:
            return None
        
        # Perform domain-specific analysis
        if image_path_or_url.startswith('http'):
            analysis = vision_client.analyze_image_by_domain(domain, image_path_or_url)
        else:
            with open(image_path_or_url, 'rb') as image_stream:
                analysis = vision_client.analyze_image_by_domain_in_stream(domain, image_stream)
        
        # Process results
        results = {}
        
        if analysis.result:
            if domain == "landmarks":
                landmarks = analysis.result.get("landmarks", [])
                if landmarks:
                    print(f"\n🏛️ LANDMARKS DETECTED:")
                    for landmark in landmarks:
                        name = landmark.get("name", "Unknown")
                        confidence = landmark.get("confidence", 0)
                        print(f"   🏛️ {name} (confidence: {confidence:.2f})")
                    results['landmarks'] = landmarks
                else:
                    print("\n🏛️ No landmarks detected")
            
            elif domain == "celebrities":
                celebrities = analysis.result.get("celebrities", [])
                if celebrities:
                    print(f"\n⭐ CELEBRITIES DETECTED:")
                    for celebrity in celebrities:
                        name = celebrity.get("name", "Unknown")
                        confidence = celebrity.get("confidence", 0)
                        face_rect = celebrity.get("faceRectangle", {})
                        print(f"   ⭐ {name} (confidence: {confidence:.2f})")
                        if face_rect:
                            print(f"      📍 Face location: ({face_rect.get('left', 0)}, {face_rect.get('top', 0)}, {face_rect.get('width', 0)}, {face_rect.get('height', 0)})")
                    results['celebrities'] = celebrities
                else:
                    print("\n⭐ No celebrities detected")
        
        return results
        
    except Exception as e:
        print(f"❌ Error in domain-specific analysis: {e}")
        return None

# Test domain-specific analysis
landmark_path = os.path.join("sample_images", "landmark.jpg")
if os.path.exists(landmark_path):
    print("\n" + "="*60)
    print("🏛️ LANDMARK DETECTION")
    print("="*60)
    landmark_results = analyze_domain_specific(landmark_path, "landmarks")

# Test celebrity detection (if faces image contains celebrities)
faces_path = os.path.join("sample_images", "faces.jpg")
if os.path.exists(faces_path):
    print("\n" + "="*60)
    print("⭐ CELEBRITY DETECTION")
    print("="*60)
    celebrity_results = analyze_domain_specific(faces_path, "celebrities")

## 7. Batch Image Processing

Process multiple images efficiently with batch operations.

In [ ]:
def batch_analyze_images(image_paths, analysis_type="description"):
    """
    Analyze multiple images in batch
    
    Args:
        image_paths (list): List of image paths or URLs
        analysis_type (str): Type of analysis ('description', 'tags', 'objects')
    
    Returns:
        list: Analysis results for all images
    """
    print(f"📊 Batch processing {len(image_paths)} images for {analysis_type} analysis...")
    
    results = []
    
    for i, image_path in enumerate(image_paths, 1):
        try:
            print(f"\n🔄 Processing image {i}/{len(image_paths)}: {os.path.basename(image_path)}")
            
            # Quick analysis based on type
            if analysis_type == "description":
                if image_path.startswith('http'):
                    analysis = vision_client.describe_image(image_path)
                else:
                    with open(image_path, 'rb') as image_stream:
                        analysis = vision_client.describe_image_in_stream(image_stream)
                
                description = analysis.captions[0].text if analysis.captions else "No description"
                confidence = analysis.captions[0].confidence if analysis.captions else 0
                
                result = {
                    'image': os.path.basename(image_path),
                    'description': description,
                    'confidence': confidence
                }
                
                print(f"   📖 Description: {description} (confidence: {confidence:.2f})")
            
            elif analysis_type == "tags":
                features = [VisualFeatureTypes.tags]
                if image_path.startswith('http'):
                    analysis = vision_client.analyze_image(image_path, visual_features=features)
                else:
                    with open(image_path, 'rb') as image_stream:
                        analysis = vision_client.analyze_image_in_stream(image_stream, visual_features=features)
                
                tags = [{'name': tag.name, 'confidence': tag.confidence} 
                       for tag in analysis.tags if tag.confidence > 0.5]
                
                result = {
                    'image': os.path.basename(image_path),
                    'tags': tags
                }
                
                print(f"   🏷️ Top tags: {', '.join([tag['name'] for tag in tags[:5]])}")
            
            results.append(result)
            
        except Exception as e:
            print(f"   ❌ Error processing {image_path}: {e}")
            results.append({
                'image': os.path.basename(image_path),
                'error': str(e)
            })
    
    # Summary
    print(f"\n📈 BATCH PROCESSING SUMMARY:")
    successful = len([r for r in results if 'error' not in r])
    failed = len([r for r in results if 'error' in r])
    print(f"   ✅ Successful: {successful}")
    print(f"   ❌ Failed: {failed}")
    
    return results

# Test batch processing with available sample images
available_images = []
for filename in sample_images.keys():
    filepath = os.path.join("sample_images", filename)
    if os.path.exists(filepath):
        available_images.append(filepath)

if available_images:
    print("\n" + "="*60)
    print("📊 BATCH IMAGE PROCESSING")
    print("="*60)
    
    # Test batch description
    batch_results_desc = batch_analyze_images(available_images, "description")
    
    print("\n" + "-"*40)
    print("🏷️ BATCH TAG ANALYSIS")
    print("-"*40)
    
    # Test batch tagging
    batch_results_tags = batch_analyze_images(available_images, "tags")
else:
    print("❌ No sample images available for batch processing")

## 8. Performance Monitoring and Best Practices

### ⚡ Performance Tips
- **Image optimization**: Resize large images before processing
- **Batch operations**: Process multiple images together
- **Caching**: Cache analysis results for frequently accessed images
- **Error handling**: Implement retry logic for transient failures

In [ ]:
import time
from datetime import datetime

def performance_test_vision(image_paths, test_type="description"):
    """
    Test Computer Vision API performance
    
    Args:
        image_paths (list): List of image paths to test
        test_type (str): Type of test to perform
    
    Returns:
        dict: Performance metrics
    """
    print(f"⚡ Performance Testing - {test_type.title()} Analysis")
    print(f"📊 Testing with {len(image_paths)} images")
    
    results = {
        'total_time': 0,
        'successful_operations': 0,
        'failed_operations': 0,
        'average_time_per_image': 0,
        'images_per_minute': 0,
        'start_time': datetime.now()
    }
    
    start_time = time.time()
    
    for i, image_path in enumerate(image_paths, 1):
        print(f"\n🔄 Processing image {i}/{len(image_paths)}...")
        image_start = time.time()
        
        try:
            if test_type == "description":
                with open(image_path, 'rb') as image_stream:
                    analysis = vision_client.describe_image_in_stream(image_stream)
                success = len(analysis.captions) > 0
            
            elif test_type == "analysis":
                features = [VisualFeatureTypes.description, VisualFeatureTypes.tags]
                with open(image_path, 'rb') as image_stream:
                    analysis = vision_client.analyze_image_in_stream(image_stream, visual_features=features)
                success = analysis.description and len(analysis.description.captions) > 0
            
            elif test_type == "ocr":
                with open(image_path, 'rb') as image_stream:
                    read_response = vision_client.read_in_stream(image_stream, raw=True)
                operation_id = read_response.headers["Operation-Location"].split("/")[-1]
                
                # Wait for completion
                while True:
                    read_result = vision_client.get_read_result(operation_id)
                    if read_result.status not in ['notStarted', 'running']:
                        break
                    time.sleep(0.1)
                
                success = read_result.status == OperationStatusCodes.succeeded
            
            image_end = time.time()
            image_time = image_end - image_start
            
            if success:
                results['successful_operations'] += 1
                print(f"✅ Completed in {image_time:.2f} seconds")
            else:
                results['failed_operations'] += 1
                print(f"❌ Failed after {image_time:.2f} seconds")
                
        except Exception as e:
            results['failed_operations'] += 1
            image_time = time.time() - image_start
            print(f"❌ Error after {image_time:.2f} seconds: {e}")
    
    end_time = time.time()
    results['total_time'] = end_time - start_time
    results['end_time'] = datetime.now()
    
    if results['successful_operations'] > 0:
        results['average_time_per_image'] = results['total_time'] / results['successful_operations']
        results['images_per_minute'] = 60 / results['average_time_per_image']
    
    # Display results
    print(f"\n📊 Performance Test Results:")
    print(f"   ⏱️ Total Time: {results['total_time']:.2f} seconds")
    print(f"   ✅ Successful: {results['successful_operations']}")
    print(f"   ❌ Failed: {results['failed_operations']}")
    print(f"   📈 Average Time/Image: {results['average_time_per_image']:.2f} seconds")
    print(f"   🚀 Images/Minute: {results['images_per_minute']:.1f}")
    print(f"   📅 Test Duration: {results['start_time'].strftime('%H:%M:%S')} - {results['end_time'].strftime('%H:%M:%S')}")
    
    return results

# Run performance tests if sample images are available
if available_images and len(available_images) >= 2:
    print("\n" + "="*60)
    print("⚡ COMPUTER VISION PERFORMANCE TESTING")
    print("="*60)
    
    # Test description performance
    desc_perf = performance_test_vision(available_images[:3], "description")
else:
    print("\n💡 Not enough sample images for performance testing")

## 9. Interactive Demo

Try Computer Vision features with your own images!

In [ ]:
# 🎯 Interactive Computer Vision Demo
# Customize these variables and run the cell!

# Option 1: Use a URL to an image
your_image_url = "https://example.com/your-image.jpg"  # Replace with your image URL

# Option 2: Use a local image path
your_local_image = "path/to/your/image.jpg"  # Replace with your local image path

# Choose analysis type
analysis_types = {
    'comprehensive': 'Full image analysis with all features',
    'description': 'Generate image description only',
    'ocr': 'Extract text from image',
    'thumbnail': 'Generate smart thumbnail',
    'objects': 'Detect objects in image'
}

selected_analysis = 'comprehensive'  # Change this to test different analyses

def interactive_analysis(image_source, analysis_type='comprehensive'):
    """
    Interactive analysis function for testing
    """
    print(f"🎯 Running {analysis_type} analysis on your image...")
    
    if analysis_type == 'comprehensive':
        return analyze_image_comprehensive(image_source)
    elif analysis_type == 'description':
        try:
            if image_source.startswith('http'):
                result = vision_client.describe_image(image_source)
            else:
                with open(image_source, 'rb') as img:
                    result = vision_client.describe_image_in_stream(img)
            
            if result.captions:
                print(f"📖 Description: {result.captions[0].text}")
                print(f"📊 Confidence: {result.captions[0].confidence:.2f}")
            return result
        except Exception as e:
            print(f"❌ Error: {e}")
            return None
    elif analysis_type == 'ocr':
        return extract_text_from_image(image_source)
    elif analysis_type == 'thumbnail':
        return generate_smart_thumbnail(image_source, 200, 200)
    
    return None

# Instructions for use
print("🎯 INTERACTIVE COMPUTER VISION DEMO")
print("=" * 50)
print("To use this demo:")
print("1. Replace 'your_image_url' or 'your_local_image' with your image")
print("2. Choose your analysis type from:")
for key, description in analysis_types.items():
    print(f"   • {key}: {description}")
print("3. Set 'selected_analysis' variable")
print("4. Uncomment the analysis code below")
print("\n💡 Available analysis types:", list(analysis_types.keys()))

# Uncomment the lines below to test with your image
# if your_image_url != "https://example.com/your-image.jpg":
#     result = interactive_analysis(your_image_url, selected_analysis)
# elif os.path.exists(your_local_image):
#     result = interactive_analysis(your_local_image, selected_analysis)
# else:
#     print("❌ Please provide a valid image URL or local path")

print("\n🚀 Ready to analyze your images!")

## 10. Troubleshooting and Best Practices

### 🔧 Common Issues and Solutions

1. **Authentication Errors**
   - ❌ "Access denied" or "Invalid subscription key"
   - ✅ Check your Computer Vision key and endpoint
   - ✅ Verify the resource is in the correct region

2. **Image Format Issues**
   - ❌ "Invalid image format"
   - ✅ Supported formats: JPEG, PNG, GIF, BMP
   - ✅ Maximum file size: 50 MB
   - ✅ Minimum dimensions: 50x50 pixels

3. **Rate Limiting**
   - ❌ "Too many requests"
   - ✅ Implement exponential backoff
   - ✅ Check your pricing tier limits

4. **OCR Accuracy**
   - ❌ Poor text recognition
   - ✅ Ensure good image quality and contrast
   - ✅ Use appropriate image resolution
   - ✅ Consider text orientation and language

### 💡 Best Practices

- **Image Quality**: Use high-resolution, well-lit images
- **Error Handling**: Always implement try-catch blocks
- **Caching**: Cache results for frequently analyzed images
- **Monitoring**: Track API usage and costs
- **Security**: Use managed identity in production

## 11. Conclusion and Next Steps

🎉 **Congratulations!** You've successfully explored Azure AI Computer Vision capabilities including:
- ✅ Comprehensive image analysis
- ✅ Object and face detection
- ✅ Optical Character Recognition (OCR)
- ✅ Smart thumbnail generation
- ✅ Domain-specific analysis (landmarks, celebrities)
- ✅ Batch processing
- ✅ Performance optimization

### 🚀 Next Steps
1. **Complete the series** with the final notebook:
   - Azure AI Language Services ✅
   - Azure AI Speech Services ✅
   - Azure AI Vision Services ✅
   - Azure AI Document Intelligence ➡️

2. **Build real applications** using Computer Vision:
   - Content moderation systems
   - Accessibility tools for visually impaired
   - Inventory management with object detection
   - Digital asset management
   - Document digitization workflows

3. **Advanced features to explore**:
   - Custom Vision for specialized models
   - Video analysis with Video Indexer
   - Form Recognizer for structured documents
   - Spatial Analysis for people counting

### 📚 Additional Resources
- [Azure Computer Vision Documentation](https://docs.microsoft.com/azure/cognitive-services/computer-vision/)
- [Computer Vision SDK Samples](https://github.com/Azure-Samples/cognitive-services-quickstart-code)
- [Vision Studio Portal](https://portal.vision.cognitive.azure.com/) - Test and customize vision models
- [Computer Vision Pricing](https://azure.microsoft.com/pricing/details/cognitive-services/computer-vision/)

### 🔗 Useful Links
- [Image Requirements](https://docs.microsoft.com/azure/cognitive-services/computer-vision/concept-image-requirements)
- [Language Support](https://docs.microsoft.com/azure/cognitive-services/computer-vision/language-support)
- [API Reference](https://docs.microsoft.com/rest/api/computervision/)
- [SDK Reference](https://docs.microsoft.com/python/api/azure-cognitiveservices-vision-computervision/)

**Happy analyzing with Azure Computer Vision! 🖼️👁️**